In [1]:
# %config IPCompleter.greedy = True

In [2]:
import os, csv, subprocess, re, random
import numpy as np
import multiprocessing, platform
# in some cases, global pool is not suggested, leading WARNINGS
# num_cores = multiprocessing.cpu_count()
# print("num of cores:", num_cores)
# global pool
# pool = multiprocessing.Pool(processes=num_cores)
# from sklearn import metrics as SKmetrics
import operator

def read_in_shakespeare():
    '''Reads in the Shakespeare dataset processesit into a list of tuples.
     Also reads in the vocab and play name lists from files.

    Each tuple consists of
    tuple[0]: The name of the play
    tuple[1] A line from the play as a list of tokenized words.

    Returns:
        tuples: A list of tuples in the above format.
        document_names: A list of the plays present in the corpus.
        vocab: A list of all tokens in the vocabulary.
    '''

    tuples = []
    with open('will_play_text.csv') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            play_name = row[1]
            line = row[5]
            line_tokens = re.sub(r'[^a-zA-Z0-9\s]', ' ', line).split()
            line_tokens = [token.lower() for token in line_tokens]
            tuples.append((play_name, line_tokens))
    with open('vocab.txt') as f: vocab = [line.strip() for line in f.readlines()]
    with open('play_names.txt') as f: document_names =  [line.strip() for line in f]
    return tuples, document_names, vocab

def get_row_vector(matrix, row_id): return matrix[row_id, :]

def get_column_vector(matrix, col_id): return matrix[:, col_id]

def compute_cosine_similarity(v1, v2): 
    '''Computes the cosine similarity of the two input vectors.
    Inputs:()
    v1: A nx1 numpy array 
    v2: A nx1 numpy array 

    Returns:
    A scalar similarity value. # a numpy array if multiple dimension
    '''
    ret = sum(np.multiply(v1, v2))/ (np.linalg.norm(v1) +np.linalg.norm(v2))
#     if 1 == len(vector1.shape) and 1 == len(vector1.shape): return ret[0] # np.double
    return ret

def compute_jaccard_similarity(vector1, vector2):
    '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
  '''  
    # http://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_similarity_score.html
    ret = np.sum(np.minimum(vector1,vector2))/(np.sum(np.maximum(vector1, vector2)))
    return ret

def compute_dice_similarity(vector1, vector2):
    '''Computes the cosine similarity of the two input vectors.

  Inputs:
    vector1: A nx1 numpy array
    vector2: A nx1 numpy array

  Returns:
    A scalar similarity value.
    '''
    ret = np.sum(np.minimum(vector1,vector2))/(np.sum(vector1) + np.sum(vector2))
    return ret

In [3]:
# cannot put it locally inside due to multiprocessing
# cannot use lambda func due to multiprocessing
def process_impv(Tuple):#         counts, Vocab = Tuple
        return [Tuple[0][w] for w in Tuple[1]]
def create_term_document_matrix(line_tuples, document_names, vocab):
    '''Returns a numpy array containing the term document matrix for the input lines.
    Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    document_names: A list of the document names
    vocab: A list of the tokens in the vocabulary
    # NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

    Let m = len(vocab) and n = len(document_names).

    Returns:
        td_matrix: A mxn numpy array where the number of rows is the number of words
          and each column corresponds to a document. A_ij contains the
          frequency with which word i occurs in document j.
    '''
    from collections import Counter, defaultdict
    Dict_doc_words_Counter = defaultdict(Counter)
    for d, wList in line_tuples: Dict_doc_words_Counter[d] += Counter(wList)
#     vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
#     docname_to_id = dict(zip(document_names, range(0, len(document_names))))
# --------------
# not parallel
    ret = [[Dict_doc_words_Counter[doc][w] for w in vocab] for doc in document_names] # fastest on Macbook
# not parallel
# --------------
# parallel
#     num_cores = multiprocessing.cpu_count()
#     print("num of cores:", num_cores)
#     pool = multiprocessing.Pool(processes=num_cores)
#     ret = pool.map(process_impv, ((Dict_doc_words_Counter[doc],vocab) for doc in document_names) ) 
# parallel
# --------------
    return np.transpose(np.array(ret))


def create_term_context_matrix(line_tuples, vocab, context_window_size=1):
    '''Returns a numpy array containing the term context matrix for the input lines.

  Inputs:
    line_tuples: A list of tuples, containing the name of the document and 
    a tokenized line from that document.
    vocab: A list of the tokens in the vocabulary

  # NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:39 PM.

  Let n = len(vocab).

  Returns:
    tc_matrix: A nxn numpy array where A_ij contains the frequency with which
        word j was found within context_window_size to the left or right of
        word i in any sentence in the tuples.
  '''
#     vocab_to_id = dict(zip(vocab, range(0, len(vocab))))
    windows_tuple = [(wList[start_pos], wList[(start_pos+1):(start_pos+context_window_size)])  \
                for d, wList in line_tuples for start_pos in range(len(wList)-context_window_size) ]
    windows_tuple += [(wList[start_pos], wList[(start_pos+1):])  \
                for d, wList in line_tuples for start_pos in range(len(wList)-context_window_size+1, len(wList)-1) ]
    from collections import Counter, defaultdict
    Dict_term_term_Counter = defaultdict(Counter)
    for w1,Lst_w2 in windows_tuple: Dict_term_term_Counter[w1] += Counter(Lst_w2)
    #
#     for d, wList in line_tuples: 
#         for start_pos in range(len(wList)-context_window_size+1):
#             Dict_term_term_Counter[wList[start_pos]] += Counter(wList[(start_pos+1):(start_pos+context_window_size)])
    ret = np.array([[Dict_term_term_Counter[w1][w2] for w2 in vocab] for w1 in vocab])
    ret += np.transpose(ret)
    return ret 

from numpy import ma
def create_PPMI_matrix(term_context_matrix):
    '''Given a term context matrix, output a PPMI matrix.
  
  See section 15.1 in the textbook.
  
  Hint: Use numpy matrix and vector operations to speed up implementation.
  
  Input:
    term_context_matrix: A nxn numpy array, where n is
        the numer of tokens in the vocab.
  
  Returns: A nxn numpy matrix, where A_ij is equal to the
     point-wise mutual information between the ith word
     and the jth word in the term_context_matrix.
  '''       
    n_words = term_context_matrix.shape[0]
    P_words_np = np.sum(term_context_matrix, axis = 0)
    Denomi_nplog2 = np.log2(np.sum(P_words_np))
    P_words_nplog2 = ma.log2(P_words_np)
    P_words_nplog2 = P_words_nplog2.filled(0)
    P_words_nplog2 = np.array([P_words_nplog2])
    ret = ma.log2(term_context_matrix)
    ret = ret.filled(0)
#     print("ret.shape",ret.shape)
#     print("np.repeat(P_words_nplog2, n_words, axis = 0)",np.repeat(P_words_nplog2, n_words, axis = 0).shape)
    ret -= np.repeat(P_words_nplog2, n_words, axis = 0)
    P_words_nplog2 = np.transpose(P_words_nplog2)
#     print("np.repeat(P_words_nplog2, n_words, axis = 1)",np.repeat(P_words_nplog2, n_words, axis = 1).shape)
    ret -= np.repeat(P_words_nplog2, n_words, axis = 1)
    ret += Denomi_nplog2
    return ret

def create_tf_idf_matrix(term_document_matrix):
  '''Given the term document matrix, output a tf-idf weighted version.

  See section 15.2.1 in the textbook.
  
  Hint: Use numpy matrix and vector operations to speed up implementation.

  Input:
    term_document_matrix: Numpy array where each column represents a document 
    and each row, the frequency of a word in that document.

  Returns:
    A numpy array with the same dimension as term_document_matrix, where
    A_ij is weighted by the inverse document frequency of document h.
  '''
  # YOUR CODE HERE
  return None

In [ ]:
# def process_impv_rk_pls(args):
#     similarity_fn,v1,v2,i = args
#     return (i,similarity_fn(v1,v2))
def rank_plays(target_play_index, term_document_matrix, similarity_fn):
    ''' Ranks the similarity of all of the plays to the target play.
  # NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:51 PM.
  Inputs:
    target_play_index: The integer index of the play we want to compare all others against.
    term_document_matrix: The term-document matrix as a mxn numpy array.
    similarity_fn: Function that should be used to compared vectors for two
      documents. Either compute_dice_similarity, compute_jaccard_similarity, or
      compute_cosine_similarity.
  Returns:
    A length-n list of integer indices corresponding to play names,
    ordered by decreasing similarity to the play indexed by target_play_index
  '''
#     print("term_document_matrix",term_document_matrix.shape)
    n_docs = term_document_matrix.shape[1]
    v1 = term_document_matrix[:,target_play_index]
    # this is a easy question, no need for parallel computing
    # -------
    # not parallel
    SimDict= dict((i,similarity_fn(v1,term_document_matrix[:,i])) for i in range(target_play_index-1))
    SimDict.update(dict((i,similarity_fn(v1,term_document_matrix[:,i])) for i in range(target_play_index+1,n_docs)))
    # not parallel
    #-------
    # parallel
#     num_cores = multiprocessing.cpu_count()
# #     print("num of cores:", num_cores)
#     pool = multiprocessing.Pool(processes=num_cores)
#     SimLst_temp = pool.map(process_impv_rk_pls, [(similarity_fn,v1,term_document_matrix[:,i],i) for i in range(target_play_index-1)] ) 
#     SimDict= dict(SimLst_temp)
#     SimLst_temp = pool.map(process_impv_rk_pls, [(similarity_fn,v1,term_document_matrix[:,i],i) for i in range(target_play_index+1,n_docs)] ) 
#     SimDict.update(dict(SimLst_temp))
    # parallel
    #-------
    L = dict(sorted(SimDict.items(), key=operator.itemgetter(1)))
    retL = list(L.keys())
    retL.reverse()
#     print("rank plays (",similarity_fn.__name__,"):", retL)
    return retL
def process_impv_rk_wds(args):
    similarity_fn,v1,v2,i = args
    return (i,similarity_fn(v1,v2))
def rank_words(target_word_index, matrix, similarity_fn):
    ''' Ranks the similarity of all of the words to the target word.
  # NOTE: THIS DOCSTRING WAS UPDATED ON JAN 24, 12:51 PM.
  Inputs:
    target_word_index: The index of the word we want to compare all others against.
    matrix: Numpy matrix where the ith row represents a vector embedding of the ith word.
    similarity_fn: Function that should be used to compared vectors for two word
      ebeddings. Either compute_dice_similarity, compute_jaccard_similarity, or
      compute_cosine_similarity.

  Returns:
    A length-n list of integer word indices, ordered by decreasing similarity to the 
    target word indexed by word_index
  '''
    n_words = matrix.shape[1]
    v1 = matrix[:,target_word_index]
    #---------------
    # not parallel
#     SimDict= dict((i,similarity_fn(v1,matrix[:,i])) for i in range(target_word_index-1))
#     SimDict.update(dict((i,similarity_fn(v1,matrix[:,i])) for i in range(target_word_index+1,n_words)))
    # not parallel
    #---------------
    num_cores = multiprocessing.cpu_count()
#     print("num of cores:", num_cores)
    pool = multiprocessing.Pool(processes=num_cores)
    SimLst_temp = pool.map(process_impv_rk_wds, [(similarity_fn,v1,matrix[:,i],i) for i in range(target_word_index-1)])
    SimDict= dict(SimLst_temp)
    SimLst_temp = pool.map(process_impv_rk_wds, [(similarity_fn,v1,matrix[:,i],i) for i in range(target_word_index+1,n_words)])
    SimDict.update(dict(SimLst_temp))
    L = dict(sorted(SimDict.items(), key=operator.itemgetter(1)))
    retL = list(L.keys())
    retL.reverse()
#     print("rank words: retL", retL)
    return retL

In [ ]:
# if __name__ == '__main__':
import time
T0 = time.time()
#-------------------------
tuples, document_names, vocab = read_in_shakespeare()
print('Computing term document matrix...')
td_matrix = create_term_document_matrix(tuples, document_names, vocab)
#     print("td_matrix",td_matrix)
print('Computing tf-idf matrix...')
tf_idf_matrix = create_tf_idf_matrix(td_matrix)

print('Computing term context matrix...')
tc_matrix = create_term_context_matrix(tuples, vocab, context_window_size=2)
print("Time elapsed:", time.time()  - T0)
print('Computing PPMI matrix...')
PPMI_matrix = create_PPMI_matrix(tc_matrix)
print("Time elapsed:", time.time()  - T0)
random_idx = random.randint(0, len(document_names)-1)
similarity_fns = [compute_cosine_similarity, compute_jaccard_similarity, compute_dice_similarity]
for sim_fn in similarity_fns:
    print('\nThe 10 most similar plays to "%s" using %s are:' % (document_names[random_idx], sim_fn.__qualname__))
    ranks = rank_plays(random_idx, td_matrix, sim_fn)
    for idx in range(0, 10):
        doc_id = ranks[idx]
        print('%d: %s' % (idx+1, document_names[doc_id]))
print("Time elapsed:", time.time()  - T0)
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on term-context frequency matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], tc_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))
print("Time elapsed:", time.time()  - T0)
word = 'juliet'
vocab_to_index = dict(zip(vocab, range(0, len(vocab))))
for sim_fn in similarity_fns:
    print('\nThe 10 most similar words to "%s" using %s on PPMI matrix are:' % (word, sim_fn.__qualname__))
    ranks = rank_words(vocab_to_index[word], PPMI_matrix, sim_fn)
    for idx in range(0, 10):
        word_id = ranks[idx]
        print('%d: %s' % (idx+1, vocab[word_id]))

print("Time elapsed:", time.time()  - T0)

Computing term document matrix...
Computing tf-idf matrix...
Computing term context matrix...
Time elapsed: 324.63192796707153
Computing PPMI matrix...
Time elapsed: 407.53529691696167

The 10 most similar plays to "Much Ado about nothing" using compute_cosine_similarity are:
1: Hamlet
2: Richard III
3: Othello
4: Cymbeline
5: Coriolanus
6: A Winters Tale
7: King Lear
8: Troilus and Cressida
9: Alls well that ends well
10: Henry IV

The 10 most similar plays to "Much Ado about nothing" using compute_jaccard_similarity are:
1: As you like it
2: Twelfth Night
3: Merchant of Venice
4: Alls well that ends well
5: Measure for measure
6: Taming of the Shrew
7: Merry Wives of Windsor
8: Othello
9: A Winters Tale
10: Romeo and Juliet

The 10 most similar plays to "Much Ado about nothing" using compute_dice_similarity are:
1: As you like it
2: Twelfth Night
3: Merchant of Venice
4: Alls well that ends well
5: Measure for measure
6: Taming of the Shrew
7: Merry Wives of Windsor
8: Othello
9: A W